# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


In this lab, I performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

some helper functions for process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, I scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
response = requests.get(static_url, headers=headers)
# Check the response status code
response.status_code

200

Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
soup.title.string
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. 


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Use the find_all function to find all table elements
html_tables = soup.find_all('table')


Starting from the third table is target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[3]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-1"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-1"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9

 see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
# Select the first launch table (Falcon 9 launches table)
# Initialize column names list
column_names = []

for th in first_launch_table.find_all('th', scope='col'):
    name = th.get_text(separator=" ", strip=True)
    if name is not None and len(name) > 0:
        column_names.append(name)



Check the extracted column names


In [12]:
print(column_names)

['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


## TASK 3: Create a data frame by parsing the launch HTML tables


create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary converted into a Pandas dataframe


In [13]:
# Create a dictionary with keys from column_names
launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column (the one with parentheses in name)
# In your case, it's 'Date and time ( UTC )'
del launch_dict['Date and time ( UTC )']

# Initialize each key with an empty list
for key in launch_dict.keys():
    launch_dict[key] = []

# Add additional columns we will extract separately
launch_dict['Version Booster'] = []  # cleaned booster version
launch_dict['Booster landing'] = []   # cleaned landing outcome
launch_dict['Date'] = []             # date only
launch_dict['Time'] = []             # time only

# Verify dictionary structure
launch_dict.keys()


dict_keys(['Flight No.', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing', 'Version Booster', 'Date', 'Time'])

In [15]:
# Normalize noisy Wikipedia column names
launch_dict['Payload'] = launch_dict.pop('Payload [ c ]')
launch_dict['Version, Booster'] = launch_dict.pop('Version, Booster [ b ]')


Next, just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, provided an incomplete code snippet below to help to fill up the `launch_dict`. 


In [16]:
# Helper function to safely get text from a cell
def get_cell_text(cell):
    """Return cleaned text from a table cell, works even if <a> tag is missing."""
    if cell is None:
        return ''
    text = ''
    if cell.a and cell.a.string:
        text = cell.a.string.strip()
    else:
        text = cell.get_text(strip=True)
    return text

# Counter for extracted rows
extracted_row = 0

for table_number, table in enumerate(
        soup.find_all('table', "wikitable plainrowheaders collapsible")):

    for rows in table.find_all("tr"):
        flag = False
        if rows.th:
            flight_number = rows.th.get_text(strip=True)
            flag = flight_number.isdigit()

        row = rows.find_all('td')

        if flag and len(row) >= 9:
            extracted_row += 1

            # Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])

            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = get_cell_text(row[1])
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_dict['Launch site'].append(get_cell_text(row[2]))

            # Payload
            launch_dict['Payload'].append(get_cell_text(row[3]))

            # Payload Mass
            launch_dict['Payload mass'].append(get_mass(row[4]))

            # Orbit
            launch_dict['Orbit'].append(get_cell_text(row[5]))

            # Customer
            launch_dict['Customer'].append(get_cell_text(row[6]))

            # Launch Outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome.strip())

            # Booster Landing
            launch_dict['Booster landing'].append(
                landing_status(row[8]).strip()
            )

print(f"Total launches extracted: {extracted_row}")


Total launches extracted: 121


After fill in the parsed launch record values into `launch_dict`, create a dataframe from it.


In [17]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

Now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 


In [20]:
df.to_csv('spacex_web_scraped.csv', index=False)


Copyright © 2021 IBM Corporation. All rights reserved.
